In [1]:
import math
import random
import igraph as ig
import pandas as pd
import numpy as np
import json
from typing import List
import os

In [2]:
class ColorUtil:
    def __init__(self):
        self.colors = ['#70f3ff', '#44cef6', '#3eede7', '#1685a9', '#177cb0', '#065279', '#003472', '#4b5cc4',
                       '#a1afc9', '#2e4e7e',
                       '#3b2e7e', '#4a4266', '#426666', '#425066', '#574266', '#8d4bbb', '#815463', '#815476',
                       '#4c221b', '#003371',
                       '#56004f', '#801dae', '#4c8dae', '#b0a4e3', '#cca4e3', '#edd1d8', '#e4c6d0', '#ff461f',
                       '#ff2d51', '#f36838',
                       '#ed5736', '#ff4777', '#f00056', '#ffb3a7', '#f47983', '#db5a6b', '#c93756', '#f9906f',
                       '#f05654', '#ff2121',
                       '#f20c00', '#8c4356', '#c83c23', '#9d2933', '#ff4c00', '#ff4e20', '#f35336', '#dc3023',
                       '#ff3300', '#cb3a56',
                       '#a98175', '#b36d61', '#ef7a82', '#ff0097', '#c32136', '#be002f', '#c91f37', '#bf242a',
                       '#c3272b', '#9d2933',
                       '#60281e', '#622a1d', '#bce672', '#c9dd22', '#bddd22', '#afdd22', '#a3d900', '#9ed900',
                       '#9ed048', '#96ce54',
                       '#00bc12', '#0eb83a', '#0eb83a', '#0aa344', '#16a951', '#21a675', '#057748', '#0c8918',
                       '#00e500', '#40de5a',
                       '#00e079', '#00e09e', '#3de1ad', '#2add9c', '#2edfa3', '#7fecad', '#a4e2c6', '#7bcfa6',
                       '#1bd1a5', '#48c0a3',
                       '#549688', '#789262', '#758a99', '#50616d', '#424c50', '#41555d', '#eaff56', '#fff143',
                       '#faff72', '#ffa631',
                       '#ffa400', '#fa8c35', '#ff8c31', '#ff8936', '#ff7500', '#ffb61e', '#ffc773', '#ffc64b',
                       '#f2be45', '#f0c239',
                       '#e9bb1d', '#d9b611', '#eacd76', '#eedeb0', '#d3b17d', '#e29c45', '#a78e44', '#c89b40',
                       '#ae7000', '#ca6924',
                       '#b25d25', '#b35c44', '#9b4400', '#9c5333', '#a88462', '#896c39', '#827100', '#6e511e',
                       '#7c4b00', '#955539',
                       '#845a33', '#ffffff', '#e9e7ef', '#f0f0f4', '#e9f1f6', '#f0fcff', '#e3f9fd', '#d6ecf0',
                       '#fffbf0', '#f2ecde',
                       '#fcefe8', '#fff2df', '#f3f9f1', '#e0eee8', '#e0f0e9', '#c0ebd7', '#bbcdc5', '#c2ccd0',
                       '#bacac6', '#808080',
                       '#75878a', '#88ada6', '#6b6882', '#725e82', '#3d3b4f', '#392f41', '#75664d', '#5d513c',
                       '#665757', '#493131',
                       '#312520', '#161823']

    def value_map(self, nums, target_min, target_max) -> list:
        '''
        将数值映射到另一个区间
        :param nums:
        :param target_min:
        :param target_max:
        :return:
        '''
        x = nums
        if type(nums) is not np.ndarray:
            x = np.array(nums)
        s_min = np.min(x)
        s_max = np.max(x)
        return list(target_min + (target_max - target_min) / (s_max - s_min) * (x - s_min))

    def value2color(self, nums, color_num) -> list:
        '''
        连续值映射为颜色
        :param nums: 值
        :param color_num: 映射的颜色数
        :return:
        '''
        x = self.value_map(nums, 1, color_num)
        colors = random.sample(self.colors, color_num + 1)
        return [colors[math.ceil(c)] for c in x]

    def category2color(self, category) -> list:
        '''
        离散值转颜色
        :param category: 离散数据
        :return:
        '''
        u_c = list(set(category))
        random.shuffle(self.colors)
        color_dict = {}

        i = 0
        color_len = len(self.colors)
        for c in u_c:
            color_dict[c] = self.colors[i]
            i += 1
            if i >= color_len:
                i = color_len - 1

        return [color_dict[c] for c in category]

工具概览,先搞出一个功能简单/能用的版本,以后再加功能

导入网络：
    1. 点击导入数据
    2. 弹出窗口,选择边/节点文件,然后点击导入有有向图/叠加网络的选项),图名字,默认文件名
    3. 后端直接读数据,然后生成网络,向前端传输表格数据 ,数据列的名字)
    4. 前端展开网络生成的详细选项,发送给后端
    5. 后端把绘图数据和各种分析数据发送到前端
    6. 前端根据数据画出图



In [3]:
class Data:
    def __init__(self, node_data: pd.DataFrame, edge_data: pd.DataFrame, relation_data: pd.DataFrame = None):
        self.node_data = node_data
        self.edge_data = edge_data
        self.relation_data = relation_data


class Style:
    def __init__(self, label: str, shape: str, size: int, color: str):
        self.label = label
        self.shape = shape
        self.size = size
        self.color = color


class DisplayOption:
    def __init__(self, graph_name: str, directed: bool, node_style: Style, edge_style: Style, layout: str = 'force'):
        self.graph_name = graph_name
        self.directed = directed
        self.node_style = node_style
        self.edge_style = edge_style
        self.layout = layout


class GraphData:
    def __init__(self):
        self.graph_name: str = None
        self.graph_count: int = 0
        self.display_options: List[DisplayOption] = []
        self.data: List[Data] = []
        self.graph: ig.Graph = None

    def new_from_dataframe(self, graph_name: str, edges: pd.DataFrame, nodes: pd.DataFrame = None,
                           directed: bool = False):
        '''
        :param graph_name:
        :param edges:
        :param nodes:
        :param directed:
        :return:
        '''
        self.__init__()

        self.data.append(Data(node_data=nodes, edge_data=edges))

        g = ig.Graph(directed=directed)

        if nodes is not None:
            # 原有id映射到新的id
            id_map = {i: g.add_vertex(name=str(i)).index for i in nodes[nodes.columns[0]]}
        else:
            id_map = {i: g.add_vertex(name=str(i)).index for i in
                      set(list(edges[edges.columns[0]] + edges[edges.columns[1]]))}

        g.add_edges(list(zip(edges[edges.columns[0]], edges[edges.columns[1]])),
                    attributes={'id': [0] * edges.shape[0]})

        graph = {
            'id': 0,  # 图id
            'id_map': {},  # 原先id到overlay中id的映射
            'node_data': None,
            'edge_data': None,
            'relation': None,
            'graph': None
        }

        # overlay网络属性
        # id(新的id，数字)    name(原先的id)    graph_id(所在的图的id)  节点
        # source(源id)     target(目标id)    graph_id(所在的图id)  边

    def overlay_from_dataframe(self, graph_name: str, edges: pd.DataFrame, nodes: pd.DataFrame = None,
                               relation: pd.DataFrame = None,
                               directed: bool = False):
        '''
        将网络叠加到现有网络中
        :param graph_name:
        :param edges:
        :param nodes:
        :param relation:
        :param directed:
        :return:
        '''

        self.graph_name += f'_{graph_name}'
        self.graph_count += 1





In [4]:
class XGraph:
    def __init__(self):
        self.color_util = ColorUtil()  # 颜色工具

        self.graph: ig.Graph = None  # 总图对象，复合复杂网络
        self.sub_graph = {}  # 子图对象map,name->{graph,node_data,edge_data}
        self.graph_info_cache = {}

    def load(self, name='', file_type: str = '', overlay: bool = False, directed: bool = False, file_path: str = '',
             node_table_path: str = '', edge_table_path: str = '', overlay_relation: str = '',
             relation_directed: bool = False):
        """
        导入数据，生成网络
        注意：节点数据第一列必须为id，字符串类型，边文件第一列为source，第二列为target


        :param name: 网络名，唯一
        :param file_type: 文件类型：table表格，other其他
        :param overlay:  是否叠加到当前网络
        :param directed:  是否有向图
        :param file_path:  file_type为other时有效
        :param node_table_path: 节点表格，仅file_type=table使有效
        :param edge_table_path:  边表格，仅file_type=table使有效
        :param overlay_relation: 网络叠加时的网络关系表格，仅overlay=true时有效
        :param relation_directed: 网络叠加时的网络关系表格是否有向边，仅overlay=true时有效
        :return:return

        # 前端网络数据导入接口
        load_info = {
            'name': '',  # 图的名字，需要唯一
            'file_type': '',  # 加载的文件类型 'table'为表格
            'overlay': '',  # 叠加到当前网络
            'directed': '',
            'file_path': '',  # type不为数据表格时有效
            'node_table_path': '',  # 节点表格的文件路径
            'edge_table_path': '',  # 边表格的文件路径
            'overlay_relation': '',  # overLay为true时有效
            'relation_directed': ''
        }

        # 加载表格数据，默认第一列是id，字符串类型，如果要叠加网络则必须保证每个文件的id都互不相同
        # 边表前两列必须为为source和target

        """
        if overlay and len(self.sub_graph) == 0:
            raise Exception("未导入网络，无法进行叠加")

        if name in self.sub_graph.keys():
            raise Exception('重复的网络名称')

        g: ig.Graph = None
        node_data: pd.DataFrame = None
        edge_data: pd.DataFrame = None
        relation_data: pd.DataFrame = None

        if file_type == 'table':
            if node_table_path != '' and os.path.exists(node_table_path):
                node_data = self.read_table(node_table_path)

            if os.path.exists(edge_table_path):
                edge_data = self.read_table(edge_table_path)

            if overlay and os.path.exists(overlay_relation):
                relation_data = self.read_table(overlay_relation)

            if node_data is None and edge_data is None:
                raise '文件都为空'

            if node_data is None:
                # 只有边文件，先生成所有节点
                node_data = pd.DataFrame()
                node_data['__id__'] = list(
                    set(edge_data[edge_data.columns[0]].astype(str)
                        + edge_data[edge_data.columns[1]].astype(str)))

            g = ig.Graph(directed=directed)
            # 修改name名称，防止重名
            node_data.columns = [f'_name' if col == 'name' else col for col in node_data.columns]

            col_names = node_data.columns
            node_id = node_data[col_names[0]].astype(str)
            # 添加节点
            for i, row in node_data.iterrows():
                kw = {col: row[col] for col in col_names[1:]}  # 其他数据
                g.add_vertex(name=node_id[i], **kw)
                # 点网络叠加操作
                if overlay and len(self.sub_graph) >= 1:
                    kw['__graph_name__'] = name  # 再复合网络中标记网络名
                    if self.graph is None:
                        self.graph = ig.Graph(directed=directed)
                    self.graph.add_vertex(name=node_id[i], **kw)

            # 添加边
            attributes = {col: edge_data[col] for col in edge_data.columns[2:]}
            edges = zip(edge_data[edge_data.columns[0]].astype(str), edge_data[edge_data.columns[1]].astype(str))
            g.add_edges(edges, attributes=attributes)

            # 边网络叠加操作
            if overlay and len(self.sub_graph) >= 1:
                # TODO:有向图和无向图混合操作，先不处理，尽量保证同一类型
                # if directed:
                #     pass

                # 向复合网络对象添加边
                # 这里变量需要重新算，不然无法添加边
                attributes = {col: edge_data[col] for col in edge_data.columns[2:]}
                attributes['__graph_name__'] = [name] * edge_data.shape[0]  # 记录边所在网络
                edges = zip(edge_data[edge_data.columns[0]].astype(str), edge_data[edge_data.columns[1]].astype(str))
                self.graph.add_edges(edges, attributes=attributes)

                # 叠加网络关系边
                if relation_data is not None:
                    attributes = {col: relation_data[col] for col in relation_data.columns[2:]}
                    attributes['__graph_name__'] = ['__none__'] * relation_data.shape[0]
                    edges = zip(relation_data[relation_data.columns[0]].astype(str),
                                relation_data[relation_data.columns[1]].astype(str))
                    self.graph.add_edges(edges, attributes=attributes)

                self.sub_graph[name] = {
                    'graph': g,
                    'node_data': node_data,
                    'edge_data': edge_data
                }

            if not overlay:
                # 非网络叠加操作，清理数据
                self.graph: ig.Graph = g.copy()
                self.graph.vs.set_attribute_values(attrname='__graph_name__', values=[name] * self.graph.vcount())
                self.graph.es.set_attribute_values(attrname='__graph_name__', values=[name] * self.graph.ecount())
                self.sub_graph = {name: {
                    'graph': g,
                    'node_data': node_data,
                    'edge_data': edge_data,
                    'option': None
                }}

    def get_data_table(self, name='') -> dict:
        '''
        获取网络对应的表格数据
        :param name: 网络名
        :return: json格式数据

        # data = {
        #     'columns': [{
        #         'title': 'a',
        #         'dataIndex': 'a'
        #     }, ],
        #     'data': [
        #         {
        #             'key': '1',
        #             'a': 44,
        #         }
        #     ],
        # }
        '''
        if name not in self.sub_graph.keys():
            raise '网络不存在'

        nodes: pd.DataFrame = self.sub_graph[name]['node_data']
        edges: pd.DataFrame = self.sub_graph[name]['edge_data']

        return {
            'nodes': self.__get_json_form_df(nodes),
            'edges': self.__get_json_form_df(edges)
        }

    def __get_json_form_df(self, df: pd.DataFrame) -> dict:
        data = {
            'columns': [{'title': col, 'dataIndex': col} for col in df.columns],
            'data': []
        }
        for i, row in df.iterrows():
            item = {
                'key': str(i)
            }
            for col in df.columns:
                item[col] = row[col]
            data['data'].append(item)
        return data

    def get_graph_info(self, name='') -> dict:
        '''
        获取网络基本属性值
        :param name: 网络名，如果为空则选定
        :return:

        '''

        g = self.graph
        if name != '':
            g = self.sub_graph[name]['graph']

        data = {
            'directed': g.is_directed(),
            'connected': g.is_connected(),
            'nodeCount': g.vcount(),
            'edgeCount': g.ecount(),
            'degree': g.degree(),
            'maxDegree': g.maxdegree(),
            'averageDegree': round(np.average(g.degree()), 4),
            'diameter': g.diameter(),
            'averagePathLen': round(g.average_path_length(), 4),
            'globalClusteringCoefficient': round(g.transitivity_undirected(), 4),  # 全局聚集系数
            'density': round(g.density(), 4),  # 网络密度

            'betweenness': [-1 if np.isnan(v) else round(v, 6) for v in g.betweenness()],  # 介数
            'closeness': [-1 if np.isnan(v) else round(v, 6) for v in g.closeness()],  # 紧密中心度
            'pageRank': [-1 if np.isnan(v) else round(v, 6) for v in g.pagerank()],  # pagerank
        }

        self.graph_info_cache[name] = data
        return data

    def render_network(self, name='', vertex_size='_度_', vertex_color='_随机_', vertex_label='_默认_',
                       vertex_shape='circle') -> dict:
        '''
        获取网络节点、边可视化数据
        :param vertex_shape:
        :param vertex_label:
        :param vertex_color: 
        :param vertex_size:
        :param name: 网络名
        :return:
        {
            "name": "",
            "vertex_size": "_度_",
            "vertex_color": "_随机_",
            "vertex_label": "_默认_",
            "vertex_shape": "circle"
        }
        '''

        if name == '_':  # 复合网络可视化
            g = self.graph
            # 要求已经可视化过所有子网
            for sub_g in self.sub_graph:
                if sub_g['option'] is None:
                    raise Exception('需要先可视化所有子网络')

            # 生成节点和边数据
            nodes = []
            edges = []

            # 网络名，用于图例筛选
            categories = []
            graph_index_map = {}
            i = 0
            for n in self.sub_graph.keys():
                categories.append({
                    "name": n
                })
                graph_index_map[n] = i
                i += 1

            for v in g.vs:
                # v: ig.Vertex
                graph_name = v['__graph_name__']
                option = self.sub_graph[graph_name]['option']
                item = {
                    "id": str(v.index),  # id 字符串
                    "name": str(option['vertex_label'][v['name']]),  # label
                    "value": '',  # 额外信息
                    "symbol": option['vertex_shape'],  # 形状
                    "symbolSize": option['vertex_size'][v['name']],  # 节点大小
                    "itemStyle": {  # 节点颜色
                        "color": option['vertex_color'][v['name']],
                    },
                    "category": graph_index_map[graph_name]
                }
                nodes.append(item)

            for e in g.es:
                # e: ig.Edge
                item = {
                    'source': str(e.source),
                    'target': str(e.target),
                }
                if e['__graph_name__'] == 'none':  # 网络关系文件
                    item['lineStyle'] = {
                        "color": "red",
                        "width": 4,
                        "type": "dashed",
                    }
                edges.append(item)

            return {
                "nodes": nodes,
                "edges": edges,
                "categories": categories,
                "directed": g.is_directed()
            }

        # 可视化子图

        # 子图的图例，如果颜色为节点属性，则将图例设置为该属性
        categories = [{"name": name}]
        color_index_map = None  # 为None则不生效

        if name != '':
            g = self.sub_graph[name]['graph']

        if name in self.graph_info_cache.keys():
            info = self.graph_info_cache[name]
        else:
            info = self.get_graph_info()

        # 计算节点样式

        if vertex_size != '':
            if vertex_size == '_默认_':
                vertex_size = None
            elif vertex_size == "_度_":
                vertex_size = [int(20 + d * 5) for d in self.color_util.value_map(info['degree'], 1, 10)]
            elif vertex_size == "_介数_":
                vertex_size = [int(20 + d * 5) for d in self.color_util.value_map(info['betweenness'], 1, 10)]
            elif vertex_size == "_紧密中心性_":
                vertex_size = [int(20 + d * 5) for d in self.color_util.value_map(info['closeness'], 1, 10)]
            else:
                if vertex_size in g.vs.attribute_names():

                    try:
                        x = np.array(g.vs.get_attribute_values(vertex_size))
                        vertex_size = [int(30 + d * 40) for d in self.color_util.value_map(x, 1, 10)]
                    except Exception as e:
                        vertex_size = None
        else:
            vertex_size = [10] * info['nodeCount']

        if vertex_color != '':
            if vertex_color.startswith('#'):
                # vertex_color = [f"#{vertex_color.split('#')[-1]}"] * self.vertex_count
                # vertex_color = [random.choice(self.colors)] * self.vertex_count
                vertex_color = [vertex_color] * info['nodeCount']
            elif vertex_color == "_随机_":
                colors = random.sample(self.color_util.colors, 20)
                vertex_color = [random.choice(colors) for _ in range(info['nodeCount'])]
            elif vertex_color == "_度_":
                vertex_color = self.color_util.value2color(info['degree'], 10)
            elif vertex_color == "_介数_":
                vertex_color = self.color_util.value2color(info['betweenness'], 10)
            elif vertex_color == "_紧密中心性_":
                vertex_color = self.color_util.value2color(info['closeness'], 10)
            else:
                if vertex_color in self.g.vs.attribute_names():
                    attr = g.vs.get_attribute_values(vertex_color)

                    try:
                        x = np.array(attr)
                        vertex_color = self.color_util.value2color(x, 10)
                    except Exception as e:
                        vertex_color = self.color_util.category2color(attr)
                        # 属性值为离散类型，此时可以更新图例，颜色映射到下标
                        color_cat_map = {}
                        for i in range(len(attr)):
                            color_cat_map[vertex_color[i]] = attr[i]

                        i = 0
                        for k, v in color_cat_map.items():
                            categories.append({
                                "name": v,
                                "itemStyle": {
                                    "color": k
                                }
                            })
                            color_index_map[k] = i
                            i += 1



        else:
            vertex_color = [random.choice(self.color_util.colors)] * info['nodeCount']

        if vertex_label == "_默认_":
            if 'name' in g.vs.attribute_names():
                vertex_label = [str(i) for i in g.vs.get_attribute_values('name')]
            else:
                vertex_label = [str(i) for i in range(info['nodeCount'])]
        else:
            if vertex_label in g.vs.attribute_names():
                vertex_label = [str(i) for i in g.vs.get_attribute_values(vertex_label)]
            else:
                vertex_label = [str(i) for i in range(info['nodeCount'])]

        # 生成节点和边数据
        nodes = []
        edges = []

        nodes = [{
            "id": str(v.index),  # id 字符串
            "name": str(vertex_label[v.index]),  # label
            "value": '',  # 额外信息
            "symbol": vertex_shape,  # 形状
            "symbolSize": vertex_size[v.index],  # 节点大小
            "itemStyle": {  # 节点颜色
                "color": vertex_color[v.index],
            },
            'category': 0 if color_index_map is None else color_index_map[vertex_color[v.index]]
        } for v in g.vs]

        for e in g.es:
            # e: ig.Edge
            item = {
                'source': str(e.source),
                'target': str(e.target),
            }
            if name == '' and e['__graph_name__'] == 'none':  # 网络关系文件
                item['lineStyle'] = {
                    "color": "red",
                    "width": 3,
                    "type": "dashed",
                }
            edges.append(item)

        # for v in g.vs:
        #     # v: ig.Vertex
        #     item = {
        #         "id": str(v.index),  # id 字符串
        #         "name": str(vertex_label[v.index]),  # label
        #         "value": '',  # 额外信息
        #         "symbol": vertex_shape,  # 形状
        #         "symbolSize": vertex_size[v.index],  # 节点大小
        #         "itemStyle": {  # 节点颜色
        #             "color": vertex_color[v.index],
        #         }
        #     }
        #     nodes.append(item)

        if name != '_':
            # 更新option信息
            self.sub_graph[name]['option'] = {
                'vertex_size': {v['name']: vertex_size[v.index] for v in g.vs},
                'vertex_shape': vertex_shape,
                'vertex_label': {v['name']: vertex_label[v.index] for v in g.vs},
                'vertex_color': {v['name']: vertex_color[v.index] for v in g.vs},
            }

        return {
            "nodes": nodes,
            "edges": edges,
            "categories": categories,
            "directed": g.is_directed()
        }

    @staticmethod
    def read_table(file_path: str) -> pd.DataFrame:
        '''
        读取表格文件
        :param file_path:
        :return:
        '''
        data = None
        try:
            if file_path.endswith('.csv'):
                data = pd.read_csv(file_path)
            else:
                data = pd.read_excel(file_path)
        except:
            raise

        return data





In [5]:
load_opt1 = {
    "name": "人物关系图",  # 图的名字，需要唯一
    "file_type": "table",  # 加载的文件类型 "table"为表格
    "overlay": False,  # 叠加到当前网络
    "directed": False,
    "file_path": "",  # type不为数据表格时有效
    "node_table_path": r"F:\Code\Python\毕业设计\网络分析\人_节点.csv",  # 节点表格的文件路径
    "edge_table_path": r"F:\Code\Python\毕业设计\网络分析\人_边.csv",  # 边表格的文件路径
    "overlay_relation": "",  # overLay为true时有效
}
load_opt2 = {
    "name": "电影关系图",  # 图的名字，需要唯一
    "file_type": "table",  # 加载的文件类型 "table"为表格
    "overlay": True,  # 叠加到当前网络
    "directed": False,
    "file_path": "",  # type不为数据表格时有效
    "node_table_path": r"F:\Code\Python\毕业设计\网络分析\电影_节点.csv",  # 节点表格的文件路径
    "edge_table_path": r"F:\Code\Python\毕业设计\网络分析\电影_边.csv",  # 边表格的文件路径
    "overlay_relation": r"F:\Code\Python\毕业设计\网络分析\关系_边.csv",  # overLay为true时有效
    "relation_directed": False
}

In [6]:
a = {
    "name": "人物关系图",
    "file_type": "table",
    "overlay": False,
    "directed": False,
    "file_path": "",
    "node_table_path": "F:/Code/Python/毕业设计/网络分析/人_节点.csv",
    "edge_table_path": "F:/Code/Python/毕业设计/网络分析/人_边.csv",
    "overlay_relation": "",
}
b = {
    "name": "电影关系图",
    "file_type": "table",
    "overlay": True,
    "directed": False,
    "file_path": "",
    "node_table_path": "F:/Code/Python/毕业设计/网络分析/电影_节点.csv",
    "edge_table_path": "F:/Code/Python/毕业设计/网络分析/电影_边.csv",
    "overlay_relation": "F:/Code/Python/毕业设计/网络分析/关系_边.csv",
    "relation_directed": False
}

In [7]:
gg = XGraph()
gg.load(**load_opt1)
gg.load(**load_opt2)

In [13]:
gg.graph.get_edge_dataframe()

,source,target,weight,__graph_name__
edge ID,,,,
0,1,32,5.0,人物关系图
1,2,37,18.0,人物关系图
2,2,67,6.0,人物关系图
3,3,52,5.0,人物关系图
4,4,60,5.0,人物关系图
...,...,...,...,...
110,42,104,1.0,__none__
111,28,108,3.0,__none__
112,58,105,3.0,__none__


In [8]:
gg.get_data_table('电影关系图')

{'nodes': {'columns': [{'title': 'MOVIE_ID', 'dataIndex': 'MOVIE_ID'},
   {'title': 'NAME', 'dataIndex': 'NAME'},
   {'title': 'ALIAS', 'dataIndex': 'ALIAS'},
   {'title': 'ACTORS', 'dataIndex': 'ACTORS'},
   {'title': 'COVER', 'dataIndex': 'COVER'},
   {'title': 'DIRECTORS', 'dataIndex': 'DIRECTORS'}],
  'data': [{'key': '0',
    'MOVIE_ID': 26670818,
    'NAME': '情定河州',
    'ALIAS': '情定临夏天使然',
    'ACTORS': '王博/吴佳尼/王姬/高丽雯/郭力行/尹哲/沈丹萍/罗中旭/臧金生/罗刚/居文沛/阎青妤',
    'COVER': nan,
    'DIRECTORS': '尹哲'},
   {'key': '1',
    'MOVIE_ID': 25815002,
    'NAME': '我不是李小龙',
    'ALIAS': nan,
    'ACTORS': '谷尚蔚/吴孟达/曾志伟/杜海涛',
    'COVER': nan,
    'DIRECTORS': '洪金宝'},
   {'key': '2',
    'MOVIE_ID': 26392287,
    'NAME': '曼哈顿中国女孩',
    'ALIAS': nan,
    'ACTORS': nan,
    'COVER': nan,
    'DIRECTORS': nan},
   {'key': '3',
    'MOVIE_ID': 26695995,
    'NAME': '绿毛水怪',
    'ALIAS': nan,
    'ACTORS': nan,
    'COVER': nan,
    'DIRECTORS': '梁栋/吴国墉'},
   {'key': '4',
    'MOVIE_ID': 26392292,
    'NAME'

In [9]:
gg.get_graph_info('')

{'directed': False,
 'connected': False,
 'nodeCount': 116,
 'edgeCount': 115,
 'degree': [2,
  2,
  2,
  1,
  1,
  1,
  5,
  1,
  2,
  2,
  2,
  1,
  3,
  1,
  2,
  3,
  2,
  4,
  3,
  2,
  2,
  4,
  1,
  2,
  2,
  0,
  5,
  0,
  3,
  1,
  2,
  2,
  1,
  2,
  2,
  0,
  2,
  3,
  3,
  1,
  2,
  4,
  3,
  3,
  2,
  2,
  1,
  1,
  2,
  1,
  2,
  2,
  8,
  2,
  4,
  4,
  0,
  0,
  2,
  0,
  3,
  0,
  2,
  1,
  3,
  1,
  0,
  3,
  2,
  1,
  0,
  3,
  0,
  0,
  0,
  1,
  2,
  0,
  5,
  1,
  0,
  1,
  3,
  2,
  1,
  2,
  0,
  2,
  1,
  1,
  2,
  2,
  3,
  4,
  4,
  3,
  2,
  3,
  2,
  3,
  2,
  1,
  4,
  1,
  4,
  3,
  2,
  1,
  3,
  1,
  2,
  4,
  3,
  2,
  0,
  3],
 'maxDegree': 8,
 'averageDegree': 1.9828,
 'diameter': 15,
 'averagePathLen': 6.2183,
 'globalClusteringCoefficient': 0.0818,
 'density': 0.0172,
 'betweenness': [88.0,
  88.0,
  0.0,
  0.0,
  0.0,
  0.0,
  345.0,
  0.0,
  72.75,
  72.75,
  0.0,
  0.0,
  260.0,
  0.0,
  2.0,
  16.866667,
  190.15,
  959.733333,
  137.8,
  88.0,

In [10]:
gg.sub_graph['电影关系图']

{'graph': <igraph.Graph at 0x128b9787b80>,
 'node_data':     MOVIE_ID              NAME       ALIAS  \
 0   26670818              情定河州     情定临夏天使然   
 1   25815002            我不是李小龙         NaN   
 2   26392287           曼哈顿中国女孩         NaN   
 3   26695995              绿毛水怪         NaN   
 4   26392292              为了祖国         NaN   
 5    6389523              天地无用         NaN   
 6   26266621             看不见的脸       見えない貌   
 7   26647079          你无法抵达的时间         NaN   
 8   26277337              黑夜之神     包公：黑夜之神   
 9   26378809               传送点         NaN   
 10  26647075           王二大爷的战争         NaN   
 11  27148159              致命复活         NaN   
 12  26528269              行星绿谷         NaN   
 13  26268576             Alpha         NaN   
 14  26680414          我和僵尸有个约会         NaN   
 15   3317352  It Is What It Is         NaN   
 16  26347315     最寒冷的冬天是旧金山的夏季         NaN   
 17  26386532               小情歌         NaN   
 18  26371294               六重门         NaN   
 19 

In [11]:
gg.render_network(**{
    "name": "人物关系图",
    "vertex_size": "_度_",
    "vertex_color": "_随机_",
    "vertex_label": "_默认_",
    "vertex_shape": "circle"
})

{'nodes': [{'id': '0',
   'name': '0',
   'value': '',
   'symbol': 'circle',
   'symbolSize': 36,
   'itemStyle': {'color': '#f20c00'}},
  {'id': '1',
   'name': '1',
   'value': '',
   'symbol': 'circle',
   'symbolSize': 36,
   'itemStyle': {'color': '#827100'}},
  {'id': '2',
   'name': '2',
   'value': '',
   'symbol': 'circle',
   'symbolSize': 36,
   'itemStyle': {'color': '#d3b17d'}},
  {'id': '3',
   'name': '3',
   'value': '',
   'symbol': 'circle',
   'symbolSize': 30,
   'itemStyle': {'color': '#3d3b4f'}},
  {'id': '4',
   'name': '4',
   'value': '',
   'symbol': 'circle',
   'symbolSize': 30,
   'itemStyle': {'color': '#9b4400'}},
  {'id': '5',
   'name': '5',
   'value': '',
   'symbol': 'circle',
   'symbolSize': 30,
   'itemStyle': {'color': '#c0ebd7'}},
  {'id': '6',
   'name': '6',
   'value': '',
   'symbol': 'circle',
   'symbolSize': 53,
   'itemStyle': {'color': '#ffffff'}},
  {'id': '7',
   'name': '7',
   'value': '',
   'symbol': 'circle',
   'symbolSize': 30

In [17]:
list(gg.sub_graph.keys())

['人物关系图', '电影关系图']

In [14]:
data:pd.DataFrame = pd.read_excel(r'C:\Users\x2845\Desktop\地铁点.xlsx')

In [20]:
data.head()

,amap_pois,station_name,amap_station_name,amap_position,amap_lines,adcode,adname
0,BV10606461,青岛站,青岛站(地铁站),"120.314543,36.064199",1号线;3号线,370202,市南区
1,BV10606457,人民会堂,人民会堂(地铁站),"120.329875,36.060331",(在建)4号线;3号线,370202,市南区
2,BV10832405,汇泉广场,汇泉广场(地铁站),"120.343300,36.059678",3号线,370202,市南区
3,BV10832403,中山公园,中山公园(地铁站),"120.351805,36.058342",3号线,370202,市南区
4,BV10832402,太平角公园,太平角公园(地铁站),"120.359654,36.053773",3号线,370202,市南区


In [34]:
data['x'] = data['amap_position'].apply(lambda x: float(str(x).split(',')[0]))
data['y']= data['amap_position'].apply(lambda x: float(str(x).split(',')[1]))

In [35]:
data.head()

,amap_pois,station_name,amap_station_name,amap_position,amap_lines,adcode,adname,x,y
0,BV10606461,青岛站,青岛站(地铁站),"120.314543,36.064199",1号线;3号线,370202,市南区,120.314543,36.064199
1,BV10606457,人民会堂,人民会堂(地铁站),"120.329875,36.060331",(在建)4号线;3号线,370202,市南区,120.329875,36.060331
2,BV10832405,汇泉广场,汇泉广场(地铁站),"120.343300,36.059678",3号线,370202,市南区,120.343300,36.059678
3,BV10832403,中山公园,中山公园(地铁站),"120.351805,36.058342",3号线,370202,市南区,120.351805,36.058342
4,BV10832402,太平角公园,太平角公园(地铁站),"120.359654,36.053773",3号线,370202,市南区,120.359654,36.053773


In [37]:
data.to_excel(r"C:\Users\x2845\Desktop\地铁点xy.xlsx",index=False)